In [1]:
import numpy as np
import pandas as pd
import time 
import datetime as datetime
import glob
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
np.random.seed(1)

In [2]:
Pad = pd.read_csv('PadData_v2.csv')
Pad.head()

,currency pair,timestamp,bid provider,bid price,bid volume,ask provider,ask price,ask volume,pad
0,AUDUSD,02.01.2019 00:00:00,LP-3,0.72714,1000000.0,LP-2,0.72718,2000000.0,0.0
1,AUDUSD,02.01.2019 00:00:01,LP-2,0.72712,2000000.0,LP-2,0.72717,2000000.0,0.0
2,AUDUSD,02.01.2019 00:00:02,LP-1,0.72713,1000000.0,LP-2,0.72718,2000000.0,0.0
3,AUDUSD,02.01.2019 00:00:03,LP-2,0.72713,1000000.0,LP-2,0.72717,2000000.0,0.0
4,AUDUSD,02.01.2019 00:00:04,LP-2,0.72713,2000000.0,LP-2,0.72717,1000000.0,0.0


In [3]:
T = 3617
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 500 # min episode length
    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    return target_bid, target_ask, other_bid, other_ask

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def get_features(target_bid, target_ask, other_bid, other_ask, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_bid.shape[1]):
        feature_span = np.append(feature_span, features(other_bid[:,i],m), axis = 1)
    for j in range(other_ask.shape[1]):
        feature_span = np.append(feature_span, features(other_ask[:,j],m), axis = 1)
    return feature_span

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
    n = np.random.randint(to_draw.shape[0] - min_history)
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    return target_bid, target_ask, feature_span

In [4]:
target_bid, target_ask, feature_span = draw_episode(16, 'AUDUSD', 1000)

In [5]:
bid, ask, features = (target_bid[16:], target_ask[16:], feature_span)

In [6]:
print(bid.shape, ask.shape, features.shape)

(3601,) (3601,) (3601, 256)


In [7]:
dbid = bid[1:]- bid[:-1]
dask = ask[1:] - ask[:-1]

In [39]:
T = 3617
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 500 # min episode length
    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    return target_bid, target_ask, other_bid, other_ask

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def get_features(target_bid, target_ask, other_bid, other_ask, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_bid.shape[1]):
        feature_span = np.append(feature_span, features(other_bid[:,i],m), axis = 1)
    for j in range(other_ask.shape[1]):
        feature_span = np.append(feature_span, features(other_ask[:,j],m), axis = 1)
    return feature_span

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
    n = np.random.randint(to_draw.shape[0] - min_history)
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    return target_bid, target_ask, feature_span
#import gym
#import gym_banana
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import Variable


#parser = argparse.ArgumentParser(description='PyTorch REINFORCE example')
# parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
#                     help='discount factor (default: 0.99)')
# parser.add_argument('--seed', type=int, default=543, metavar='N',
#                     help='random seed (default: 543)')
# parser.add_argument('--render', action='store_true',
#                     help='render the environment')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='interval between training status logs (default: 10)')
#args = parser.parse_args()


#env = gym.make('Banana-v0')
#env.seed(1)
torch.manual_seed(1)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(p=0.6)
#         self.affine2 = nn.Linear(128, 1)
        self.mu = nn.Linear(1,1, bias = True)
        #self.affine1 = nn.Linear(1, 3)
        self.tanh = nn.Tanh()

        self.saved_log_probs = []
        self.rewards = []
        self.actions = []

    def forward(self, x, y):
        x = self.affine1(x)
        x = self.dropout(x)
#         print(x.size())
        mu_sigma = self.mu(y)
#         print(mu_sigma.size())
        action = self.tanh(x + mu_sigma)
#         action_scores = self.affine2(x)
#         return F.softmax(action_scores, dim=1)
        return action


policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()
c = 0


def select_action(state, previous_action):
    state = torch.from_numpy(state).float().unsqueeze(0)
    previous_action = torch.from_numpy(np.array([previous_action])).float().unsqueeze(0)
    prob = policy(state, previous_action).item()
   # print(prob)
    action = min(abs(prob+1),abs(prob),abs(prob-1))-1
    #policy.saved_log_probs.append(m.log_prob(action))
    return action


def finish_episode():
    R = 0
    policy_loss = []
    returns = []
    for r in policy.rewards[::-1]:
        R = r
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
  #  for log_prob, R in zip(policy.saved_log_probs, returns):
        #print("logprob", log_prob)
        #print("R", R)
       # policy_loss.append(-log_prob * R)
    optimizer.zero_grad()
  #  policy_loss = torch.cat(policy_loss).sum()
    policy_loss = returns.sum()
    policy_loss = Variable(policy_loss, requires_grad = True)
    print(policy_loss)
    policy_loss.backward(retain_graph=True)
    optimizer.step()
    print ("Finished one episode and the policy loss is {}".format(policy_loss))
    #del policy.rewards[:]
    #del policy.saved|_log_probs[:]


def main():
    running_reward = 10

    for i_episode in count(100):
        target_bid, target_ask, feature_span = draw_episode(16, 'AUDUSD', 1000)
        bid, ask, features = target_bid[16:], target_ask[16:], feature_span
        previous_action = 0
        for t in range(3600):  # Don't infinite loop while learning
            state = feature_span[t]
            action = select_action(state, previous_action)
            reward = (action<0)*(ask[t+1] - ask[t]) + (action>0)*(bid[t+1] - bid[t]) - c*np.abs(action-previous_action)
            #reward -= 0.01 * abs(previous_action - action)  
            # the current price is 
#             state, reward, done, _ = env.step(action) #change banana
  #          if args.render:
  #              env.render()
            policy.rewards.append(reward)
#             ep_reward += reward
            policy.actions.append(action)
            previous_action = action


#         running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode()
#         if i_episode % 5 == 0:
#             print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
#                   i_episode, ep_reward, running_reward))
#         if running_reward > env.spec.reward_threshold:
#             print("Solved! Running reward is now {} and "
#                   "the last episode runs to {} time steps!".format(running_reward, t))
#             break


if __name__ == '__main__':
    main()

tensor(-5.3088e-05, requires_grad=True)
Finished one episode and the policy loss is -5.308818072080612e-05
tensor(-9.5367e-07, requires_grad=True)
Finished one episode and the policy loss is -9.5367431640625e-07
tensor(-6.1495e-05, requires_grad=True)
Finished one episode and the policy loss is -6.149522960186005e-05
tensor(-0.0003, requires_grad=True)
Finished one episode and the policy loss is -0.0002918243408203125
tensor(-5.2433e-06, requires_grad=True)
Finished one episode and the policy loss is -5.243346095085144e-06
tensor(-0.0006, requires_grad=True)
Finished one episode and the policy loss is -0.0006389617919921875
tensor(-0.0022, requires_grad=True)
Finished one episode and the policy loss is -0.002188989892601967
tensor(0.0017, requires_grad=True)
Finished one episode and the policy loss is 0.00168609619140625
tensor(-0.0013, requires_grad=True)
Finished one episode and the policy loss is -0.001278810203075409
tensor(-0.0020, requires_grad=True)
Finished one episode and the 

KeyboardInterrupt: 